# Convert quantum results to GPT3 strings

- Count total shots
- Filter errors (eg: '00110', '00000')
- Add missing zero results when they are not present
- Convert qbit binary string back to index
- Order results by original places index
- Map results to places and convert to probabilities
- Output well formatted likelyhood strings for the GPT3 API

## Quantum results cleanup and post processing

In [28]:
# Numpy is needed for the entropy computation
import numpy as np

# JSON data sent to the server
json_data = {
    "computer": "ionq",  # quantum computer to use
    "places": [          # list of places to visit and their probabilities
        ["Gym", 0.2],
        ["Opera", 0.5],
        ["Rooftop bar", 0.9],
        ["Street fair", 0.0],
        ["Pool", 0.1],
    ]
}
places = json_data['places']
qbit_count = len(places)

# Quantum computer results (generating missing keys and different kind of errors)
results = {
  "10000": 85,  # 4
  "00001": 836, # 0
  "00010": 99,  # 1
  "01000": 1,   # 3
  # "00100": 0, # 2 has no result
  "01100": 2,   # Error
  "00000": 1,   # Error
}

# Total shots
total_shots = sum(results.values())

In [29]:
def filter_errors(result):
    ''' Detect errors and remove it from the results '''
    return {key: value for key, value in result.items() if key.count('1') == 1}

# Filtered results
filtered_results = filter_errors(results)
print(filtered_results)

{'10000': 85, '00001': 836, '00010': 99, '01000': 1}


In [30]:
def convert_binary_to_index(binary_string):
    ''' Convert binary strings to indices (e.g. '00010' -> 1 and '10000' -> 4) '''
    if '1' in binary_string:
        return len(binary_string) - binary_string.index('1') - 1
    else:
        raise ValueError('Binary string must contain at least one 1')

index0 = convert_binary_to_index('00001')
print(index0)
index2 = convert_binary_to_index('00100')
print(index2)

0
2


In [31]:
def convert_index_to_binary(index, qbit_count):
    ''' Convert index to binary string (e.g. 3 -> '01000' and 4 -> '10000') '''
    zero_str = "0" * qbit_count
    zero_str = zero_str[:index] + "1" + zero_str[index + 1:]
    return zero_str[::-1]

binary_string0 = convert_index_to_binary(0, 5)
print(binary_string0)
binary_string1 = convert_index_to_binary(1, 5)
print(binary_string1)
binary_string3 = convert_index_to_binary(3, 5)
print(binary_string3)

00001
00010
01000


In [32]:
def add_missing_results(result, qbit_count):
    ''' Add missing results to the result dictionary '''
    for i in range(qbit_count):
        binary_string = convert_index_to_binary(i, qbit_count)
        if binary_string not in result:
            result[binary_string] = 0
    return result

completed_results = add_missing_results(filtered_result, qbit_count)
print(completed_results)

{'10000': 85, '00001': 836, '00010': 99, '01000': 1, '00100': 0}


In [33]:
def order_results(result):
    ''' Order results using the original index '''
    ordered_result = []
    for key, value in result.items():
        index = convert_binary_to_index(key)
        ordered_result.append([index, key, value])
    ordered_result.sort()
    return ordered_result

ordered_results = order_results(completed_results)
print(ordered_results)

[[0, '00001', 836], [1, '00010', 99], [2, '00100', 0], [3, '01000', 1], [4, '10000', 85]]


In [34]:
def convert_to_places(result, places, total_shots):
    ''' Convert qbit keys and shots to places and probabilities (e.g. [1, '00010', 85] -> ['Opera', 0.085]) '''
    readable_result = []
    for line in result:
        place = places[line[0]][0]
        probability = line[2] / total_shots
        readable_result.append([place, probability])
    return readable_result

processed_results = convert_to_places(ordered_results, places, total_shots)
print(processed_results)

[['Gym', 0.81640625], ['Opera', 0.0966796875], ['Rooftop bar', 0.0], ['Street fair', 0.0009765625], ['Pool', 0.0830078125]]


In [35]:
def convert_probability_to_likelyhood(probability):
    ''' Method to convert probabilities to likelyhoods strings.'''
    if 0 <= probability < 0.25:
        return "wasn't at the "
    elif 0.25 <= probability < 0.75:
        return "may have gone to the "
    elif 0.75 <= probability <= 1.0:
        return "likely went to the "
    else:
        raise ValueError('Probability must be between 0 and 1.')

# Convert probabilities to readable GPT3 strings
likelyhood = convert_probability_to_likelyhood(0.6)
place = 'opera'
worded_result = likelyhood + place
print(worded_result)

may have gone to the opera


## Shot count and percentage

In [37]:
# Total shots
total_shots = sum(results.values())

# Valid shots from the filtered results
valid_shots = sum(filtered_results.values())

# Compute error shots
error_shots = total_shots - valid_shots

# Compute error percentage
error_percentage = error_shots / total_shots

print(f'Error percentage: {error_shots} / {total_shots} -> {error_percentage:.2%}')

Error percentage: 3 / 1024 -> 0.29%


## Shannon's entropy

We can use Shannon's entropy to measure the uncertainty of a probability distribution. 

The higher the entropy, the more uncertain and 'chaotic' the distribution is.

In [41]:
def entropy(probabilities):
    '''Compute the normed entropy of the probability distribution.'''
    if isinstance(probabilities, list):
        probabilities = np.array(probabilities)
    scaled = -probabilities * np.log(probabilities)
    scaled[np.isnan(scaled)] = 0

    entropy_max = np.log(len(probabilities))
    entropy_val = np.sum(scaled)
    return entropy_val / entropy_max

probabilities = [line[1] for line in processed_results]
entropy_value = entropy(probabilities)
print(f'Normed entropy: {entropy(probabilities):.2f}')

Normed entropy: 0.38


c:\Users\Windows\.virtualenvs\quintessence--qpE3CGX\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """
c:\Users\Windows\.virtualenvs\quintessence--qpE3CGX\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in multiply
  """


In [43]:

def convert_entropy_to_words(normed_entropy):
    '''Convert the entropy value to a word.'''
    if 0 <= normed_entropy < 0.1:
        return "uneventful"
    elif 0.1 <= normed_entropy < 0.4:
        return "boring"
    elif 0.4 <= normed_entropy < 0.6:
        return "regular"
    elif 0.6 <= normed_entropy < 0.9:
        return "exciting"
    elif 0.9 <= normed_entropy < 1.0:
        return "chaotic"
    else:
        raise ValueError('Entropy must be between 0 and 1.')

print(f'Entropy string: {convert_entropy_to_words(entropy_value)}')

Entropy string: boring


## Create GPT3 prompt

Use entropy and quantum results to create a GPT3 prompt.

In [50]:
def create_gpt3_prompt(results, entropy):
    '''Method to create the GPT-3 prompt.'''

    # Initial place
    initial_place = results[0][0]

    # Convert numerical values to words
    entropy_adjective = convert_entropy_to_words(entropy)

    # Sort top 3 results
    results.sort(key=lambda x: x[1], reverse=True)
    results = results[:3]
    print(results)

    # Convert probabilities to readable GPT3 strings
    worded_results = []
    for result in results:
        likelyhood = convert_probability_to_likelyhood(result[1])
        place = result[0]
        worded_results.append(likelyhood + place)

    prompt = '''Mr. Quanta cannot remember how he got here. Tell the story of him trying to
                remember how he got here in 3 steps and be descriptive.
                He only remembers a few things, and considers each possible place one at a time. 
                Use grandiose language. Embelish everything and paint a picture with words. 
                Make the descriptions drip with imagery. '''
    prompt += f"He had a { entropy_adjective } time before "
    prompt += f"awakening at the { initial_place }, and before that "

    # Append probability strings in the prompt
    for worded_result in worded_results:
        prompt += worded_result  + ', '
    prompt += "."

    return prompt

# Create GPT-3 prompt
prompt = create_gpt3_prompt(processed_results, entropy_value)
print(prompt)

[['Gym', 0.81640625], ['Opera', 0.0966796875], ['Pool', 0.0830078125]]
Mr. Quanta cannot remember how he got here. Tell the story of him trying to
                remember how he got here in 3 steps and be descriptive.
                He only remembers a few things, and considers each possible place one at a time. 
                Use grandiose language. Embelish everything and paint a picture with words. 
                Make the descriptions drip with imagery. He had a boring time before awakening at the Gym, and before that likely went to the Gym, wasn't at the Opera, wasn't at the Pool, .
